### Macro Architecture 
First, we define the ResNet-like Macro architecture. 

**The goal of this macro-architecture is to represent all resnet-like architectures, including Wide-Resnets and ResNext.**

The Macro-architecture is defined as illustrated in the following figure. 

<img src="resnet_like.png" alt="macro-architecture" width="80%"/>

Search Space size: >8000000

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
class ResidualBranch(nn.Module):
    def __init__(self, in_channels, out_channels, filter_size, stride, branch_index):
        super(ResidualBranch, self).__init__()

        self.residual_branch = nn.Sequential()

        self.residual_branch.add_module('Branch_{}:ReLU_1'.format(branch_index),
                                        nn.ReLU(inplace=False))
        self.residual_branch.add_module('Branch_{}:Conv_1'.format(branch_index),
                                        nn.Conv2d(in_channels,
                                                  out_channels,
                                                  kernel_size=filter_size,
                                                  stride=stride,
                                                  padding=round(filter_size / 3),
                                                  bias=False))
        self.residual_branch.add_module('Branch_{}:BN_1'.format(branch_index),
                                        nn.BatchNorm2d(out_channels))
        self.residual_branch.add_module('Branch_{}:ReLU_2'.format(branch_index),
                                        nn.ReLU(inplace=False))
        self.residual_branch.add_module('Branch_{}:Conv_2'.format(branch_index),
                                        nn.Conv2d(out_channels,
                                                  out_channels,
                                                  kernel_size=filter_size,
                                                  stride=1,
                                                  padding=round(filter_size / 3),
                                                  bias=False))
        self.residual_branch.add_module('Branch_{}:BN_2'.format(branch_index),
                                        nn.BatchNorm2d(out_channels))

    def forward(self, x):
        return self.residual_branch(x)

In [12]:
class SkipConnection(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(SkipConnection, self).__init__()

        self.s1 = nn.Sequential()
        self.s1.add_module('Skip_1_AvgPool',
                           nn.AvgPool2d(1, stride=stride))
        self.s1.add_module('Skip_1_Conv',
                           nn.Conv2d(in_channels,
                                     int(out_channels / 2),
                                     kernel_size=1,
                                     stride=1,
                                     padding=0,
                                     bias=False))

        self.s2 = nn.Sequential()
        self.s2.add_module('Skip_2_AvgPool',
                           nn.AvgPool2d(1, stride=stride))
        self.s2.add_module('Skip_2_Conv',
                           nn.Conv2d(in_channels,
                                     int(out_channels / 2) if out_channels % 2 == 0 else int(out_channels / 2) + 1,
                                     kernel_size=1,
                                     stride=1,
                                     padding=0,
                                     bias=False))

        self.batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out1 = F.relu(x, inplace=False)
        out1 = self.s1(out1)

        out2 = F.pad(x[:, :, 1:, 1:], (0, 1, 0, 1))
        out2 = self.s2(out2)

        out = torch.cat([out1, out2], dim=1)
        out = self.batch_norm(out)

        return out


class BasicBlock(nn.Module):
    def __init__(self, n_input_plane, n_output_plane, filter_size, res_branches, stride):
        super(BasicBlock, self).__init__()

        self.branches = nn.ModuleList([ResidualBranch(n_input_plane, n_output_plane, filter_size, stride, branch + 1) for branch in range(res_branches)])

        self.skip = nn.Sequential()
        if n_input_plane != n_output_plane or stride != 1:
            self.skip.add_module('Skip_connection',
                                 SkipConnection(n_input_plane, n_output_plane, stride))
                                 

    def forward(self, x):
        out = sum([self.branches[i](x) for i in range(len(self.branches))])
        return out + self.skip(x)


class ResidualGroup(nn.Module):
    def __init__(self, block, n_input_plane, n_output_plane, n_blocks, filter_size, res_branches, stride):
        super(ResidualGroup, self).__init__()
        self.group = nn.Sequential()
        self.n_blocks = n_blocks

        self.group.add_module('Block_1',
                              block(n_input_plane,
                                    n_output_plane,
                                    filter_size,
                                    res_branches,
                                    stride=stride))

        # The following residual block do not perform any downsampling (stride=1)
        for block_index in range(2, n_blocks + 1):
            block_name = 'Block_{}'.format(block_index)
            self.group.add_module(block_name,
                                  block(n_output_plane,
                                        n_output_plane,
                                        filter_size,
                                        res_branches,
                                        stride=1))

    def forward(self, x):
        return self.group(x)


class Network(nn.Module):
    def __init__(self, config):
        super(Network, self).__init__()

        self.M = config["M"]
        self.residual_blocks = {'Group_1': config["R1"],
                                'Group_2': config["R2"],
                                'Group_3': config["R3"],
                                'Group_4': config["R4"],
                                'Group_5': config["R5"]
                                }
 
        self.widen_factors = {'Group_1': config["widenfact0"],
                              'Group_2': config["widenfact1"],
                              'Group_3': config["widenfact2"],
                              'Group_4': config["widenfact3"],
                              'Group_5': config["widenfact4"]
                              }

        self.res_branches = {'Group_1': config["B0"],
                             'Group_2': config["B1"],
                             'Group_3': config["B2"],
                             'Group_4': config["B3"],
                             'Group_5': config["B4"]
                             }

        self.conv_blocks =  {'Group_1': config["convblock0"],
                             'Group_2': config["convblock1"],
                             'Group_3': config["convblock2"],
                             'Group_4': config["convblock3"],
                             'Group_5': config["convblock4"]
                             }
        
        self.filters_size = {'Group_1': 3,
                             'Group_2': 3,
                             'Group_3': 3,
                             'Group_4': 3,
                             'Group_5': 3
                             }
        
        self.model = nn.Sequential()
        block = BasicBlock
        self.model.add_module('Conv_0',
                              nn.Conv2d(3,
                                        config["out_channel0"],
                                        kernel_size=3,
                                        stride=1,
                                        padding=1,
                                        bias=False))
        
        self.model.add_module('BN_0',
                              nn.BatchNorm2d(config["out_channel0"]))

        feature_maps_in = int(round(config["out_channel0"] * self.widen_factors['Group_1']))
        self.model.add_module('Group_1',
                              ResidualGroup(block, 
                                            config["out_channel0"], 
                                            feature_maps_in, 
                                            self.residual_blocks['Group_1'], 
                                            self.filters_size['Group_1'],
                                            self.res_branches['Group_1'],
                                            1))
        feature_maps_out = int(round(feature_maps_in * self.widen_factors['Group_2']))
        for m in range(2, self.M + 1):
            self.model.add_module('Group_{}'.format(m),
                                  ResidualGroup(block, 
                                                feature_maps_in, 
                                                feature_maps_out, 
                                                self.residual_blocks['Group_{}'.format(m)],
                                                self.filters_size['Group_{}'.format(m)],
                                                self.res_branches['Group_{}'.format(m)],
                                                2 if m in (self.M, self.M - 1) else 1))
            feature_maps_in = feature_maps_out

        self.feature_maps_out = feature_maps_out
        self.model.add_module('ReLU_0',
                              nn.ReLU(inplace=True))
        self.model.add_module('AveragePool',
                              nn.AvgPool2d(8, stride=1))
        self.fc = nn.Linear(feature_maps_out, 10)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, self.feature_maps_out)
        x = self.fc(x)
        return x

In [15]:
from config_space import ConfigSpace
CS = ConfigSpace("VWW") 
config = CS.sample_arch()
network = Network(config)

print(config)

{'out_channel0': 17, 'M': 3, 'R1': 11, 'R2': 2, 'R3': 3, 'R4': 14, 'R5': 0, 'convblock0': 1, 'widenfact0': 0.511255906439347, 'B0': 2, 'convblock1': 1, 'widenfact1': 0.7990545759894023, 'B1': 1, 'convblock2': 1, 'widenfact2': 0.6171399501335335, 'B2': 4, 'convblock3': 1, 'widenfact3': 0.741254648781051, 'B3': 4, 'convblock4': 1, 'widenfact4': 0.7114603245948148, 'B4': 3}


In [16]:
network

Network(
  (model): Sequential(
    (Conv_0): Conv2d(3, 17, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (BN_0): BatchNorm2d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Group_1): ResidualGroup(
      (group): Sequential(
        (Block_1): BasicBlock(
          (branches): ModuleList(
            (0): ResidualBranch(
              (residual_branch): Sequential(
                (Branch_1:ReLU_1): ReLU()
                (Branch_1:Conv_1): Conv2d(17, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (Branch_1:BN_1): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (Branch_1:ReLU_2): ReLU()
                (Branch_1:Conv_2): Conv2d(9, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (Branch_1:BN_2): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
            )
            (1): Residua

In [17]:
# test model 
dummy_input = torch.randn([1, 3, 100, 100])
network.eval()
np_torch_out = network(dummy_input).data.numpy()
print(np_torch_out)

[[ 0.04763719  0.22375715 -0.17803589 ...  0.06808586  0.06999959
   0.05319611]
 [ 0.04760651  0.22372478 -0.17807284 ...  0.06808644  0.0700351
   0.05318892]
 [ 0.04765865  0.22371954 -0.1779406  ...  0.06810446  0.06998012
   0.05318211]
 ...
 [ 0.11716738  0.25919074 -0.13698131 ...  0.0383015   0.03163335
   0.09851553]
 [ 0.10686804  0.2544343  -0.13901041 ...  0.04080017  0.04133644
   0.10287935]
 [ 0.1246848   0.26833233 -0.14009991 ...  0.03811509  0.03042329
   0.1031388 ]]


In [19]:
import pandas as pd

In [40]:
# Sample Architectures from search space 

configs =  []
for i in range(1000):
    config = CS.sample_arch()
    configs.append(config)
    
ss = pd.DataFrame(configs)
    
ss.head()

,out_channel0,M,R1,R2,R3,R4,R5,convblock0,widenfact0,B0,...,B1,convblock2,widenfact2,B2,convblock3,widenfact3,B3,convblock4,widenfact4,B4
0,47,3,13,8,11,1,0,1,0.796181,4,...,2,1,0.708632,2,1,0.653344,3,1,0.784293,4
1,25,2,11,12,2,7,14,1,0.515494,1,...,2,1,0.581311,3,1,0.570107,4,1,0.534046,3
2,60,2,14,14,8,9,0,1,0.751199,3,...,4,1,0.730001,1,1,0.703210,3,1,0.638999,4
3,52,1,14,14,5,1,1,1,0.676028,3,...,1,1,0.695805,4,1,0.630569,3,1,0.568021,1
4,27,3,9,14,2,7,8,1,0.781097,2,...,1,1,0.778140,1,1,0.708540,3,1,0.564992,2


In [41]:
pd.to_csv(ss)

AttributeError: module 'pandas' has no attribute 'to_csv'